In [1]:
import tensorflow.keras
from keras.layers import Dense,MaxPool2D,Conv2D,Flatten,GlobalAveragePooling2D,Dropout
from keras.models import Sequential,Model
from keras.utils import np_utils
from keras.datasets import cifar10
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np
from tensorflow.keras.optimizers import Adam
from keras.applications.xception import Xception,preprocess_input
import cv2

In [2]:
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()

print("xtrain:",xtrain.shape,"ytrain:",ytrain.shape)
print("xtest:",xtest.shape,"ytest:",ytest.shape)

#one hot encoding
ytrain=np_utils.to_categorical(ytrain)
ytest=np_utils.to_categorical(ytest)

#preprocessing images

xtrain=np.array([cv2.resize(xtrain[i],dsize=(72,72),interpolation=cv2.INTER_CUBIC) for i in range(0,len(xtrain))]).astype('float16')
xtest=np.array([cv2.resize(xtest[i],dsize=(72,72),interpolation=cv2.INTER_CUBIC) for i in range(0,len(xtest))]).astype('float16')

xtrain=preprocess_input(xtrain)
xtest=preprocess_input(xtest)

print("xtrain:",xtrain.shape,"ytrain:",ytrain.shape)
print("xtest:",xtest.shape,"ytest:",ytest.shape)





170500096/170498071 [==============================] - 4s 0us/step
xtrain: (50000, 32, 32, 3) ytrain: (50000, 1)
xtest: (10000, 32, 32, 3) ytest: (10000, 1)
xtrain: (50000, 72, 72, 3) ytrain: (50000, 10)
xtest: (10000, 72, 72, 3) ytest: (10000, 10)


In [3]:
# using pretrained model

tl_model=Xception(include_top=False,input_shape=(72,72,3),weights='imagenet')#tl_model means transfered learnt model

for layer in tl_model.layers:
    layer.trainable = False
tl_model.summary()



"""def batch(i):
  t=np.array([cv2.resize(xtrain[i*500+j],dsize=(72,72),interpolation=cv2.INTER_CUBIC) for j in range(0,500)]).astype('float16')
  t=preprocess_input(t)
  return tl_model.predict(t)





for i in range(0,5):
  temp=batch(i)
  print(temp.shape,"temp shape")
  if i==0:
    K=temp
    print('local if')
    print(id(K),'\n')
  else :
    K=np.concatenate((K,temp),axis=-1)
    xt=K
    print('local else')

    print(id(xt),'\n')
  #print(xt)


#xtrain=tl_model.predict(xtrain)
#xtest=tl_model.predict(xtest)""" #

83689472/83683744 [==============================] - 0s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 72, 72, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
__________________________

'def batch(i):\n  t=np.array([cv2.resize(xtrain[i*500+j],dsize=(72,72),interpolation=cv2.INTER_CUBIC) for j in range(0,500)]).astype(\'float16\')\n  t=preprocess_input(t)\n  return tl_model.predict(t)\n\n\n\n\n\nfor i in range(0,5):\n  temp=batch(i)\n  print(temp.shape,"temp shape")\n  if i==0:\n    K=temp\n    print(\'local if\')\n    print(id(K),\'\n\')\n  else :\n    K=np.concatenate((K,temp),axis=-1)\n    xt=K\n    print(\'local else\')\n\n    print(id(xt),\'\n\')\n  #print(xt)\n\n\n#xtrain=tl_model.predict(xtrain)\n#xtest=tl_model.predict(xtest)'

In [ ]:
"""
#making ml task classifier
model=Sequential()
model.add(GlobalAveragePooling2D(input_shape=(7,7,2048)))
model.add(Dropout(0.3))
model.add(Dense(units=ytest.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])"""


In [4]:
flat = Flatten()(tl_model.layers[-1].output)
d1 = Dense(1024, activation='relu')(flat)
d2 = Dropout(0.4)(d1)
output = Dense(10, activation='softmax')(d2)
# define new model
model = Model(inputs=tl_model.inputs, outputs=output)

model.compile(optimizer = Adam(lr = 0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])


training_history = None
batch_size = 128
epochs = 10
training_history =  model.fit(
    xtrain,
    ytrain,
    validation_data = (xtest,ytest),
    batch_size = batch_size,
    epochs = epochs,
    shuffle = True,
    verbose = 1
)


Epoch 1/10
391/391 [==============================] - 55s 59ms/step - loss: 2.3035 - acc: 0.6249 - val_loss: 0.7729 - val_acc: 0.7373
Epoch 2/10
391/391 [==============================] - 21s 54ms/step - loss: 0.7795 - acc: 0.7377 - val_loss: 0.7275 - val_acc: 0.7522
Epoch 3/10
391/391 [==============================] - 21s 54ms/step - loss: 0.7144 - acc: 0.7550 - val_loss: 0.6937 - val_acc: 0.7629
Epoch 4/10
391/391 [==============================] - 21s 54ms/step - loss: 0.6629 - acc: 0.7708 - val_loss: 0.6910 - val_acc: 0.7678
Epoch 5/10
391/391 [==============================] - 21s 54ms/step - loss: 0.6188 - acc: 0.7849 - val_loss: 0.6855 - val_acc: 0.7680
Epoch 6/10
391/391 [==============================] - 21s 54ms/step - loss: 0.5921 - acc: 0.7930 - val_loss: 0.6888 - val_acc: 0.7656
Epoch 7/10
391/391 [==============================] - 21s 54ms/step - loss: 0.5637 - acc: 0.8001 - val_loss: 0.7052 - val_acc: 0.7667
Epoch 8/10
391/391 [==============================] - 21s 54ms

In [5]:
prediction = model.predict(xtest)
ytest = np.argmax(ytest, axis=1)
prediction = np.argmax(prediction, axis=1)
print('\nAccuracy:', accuracy_score(ytest, prediction, normalize=True))
print('\nClassification Report:' + classification_report(ytest, prediction))
print('\n Confusion Matrix:\n',confusion_matrix(ytest, prediction))


Accuracy: 0.7728

Classification Report:              precision    recall  f1-score   support

           0       0.75      0.82      0.78      1000
           1       0.91      0.83      0.87      1000
           2       0.80      0.68      0.73      1000
           3       0.61      0.67      0.64      1000
           4       0.71      0.70      0.71      1000
           5       0.76      0.69      0.72      1000
           6       0.75      0.81      0.78      1000
           7       0.79      0.81      0.80      1000
           8       0.84      0.86      0.85      1000
           9       0.83      0.87      0.85      1000

    accuracy                           0.77     10000
   macro avg       0.78      0.77      0.77     10000
weighted avg       0.78      0.77      0.77     10000


 Confusion Matrix:
 [[816  14  19  17  12   3   4  15  77  23]
 [ 21 827   2  15   1   5   1   1  21 106]
 [ 53   2 679  52  78  27  78  24   5   2]
 [ 21   6  30 665  52 105  68  37   8   8]
 [ 21  